In [26]:
!pip install python-dotenv

In [32]:
import os
from dotenv import load_dotenv
# from utils import get_openai_api_key
load_dotenv()
# openai_api_key = get_openai_api_key()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# OPENAI_API_KEY='sk-proj-KUKJxZAe94aPIf6FqfRCT3BlbkFJaUcriBsidGd36NghCE82'

# This is the YouTube video we're going to use.
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"

print(OPENAI_API_KEY)

sk-proj-2M9GUFjFraxWDhckt63PT3BlbkFJYmrzit7HKxSJmgsQZSHy


In [28]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [24]:
# Checking the model is working
model.invoke("who won 2011 cricket world cup")

AIMessage(content='India won the 2011 Cricket World Cup.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 15, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-329eb511-0f50-467b-8300-7b1d3021516d-0')

sk-proj-2M9GUFjFraxWDhckt63PT3BlbkFJYmrzit7HKxSJmgsQZSHy
